In [101]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import json


In [102]:
load_dotenv(override=True)
OPENAI_API_KEY = os.getenv('GEMINI_API_KEY')


In [103]:
client = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/", api_key = OPENAI_API_KEY)

In [104]:
response = client.chat.completions.create(model = "gemini-2.5-flash",messages={'role':'user','content':'hellow'})

In [105]:
print(response.choices[0].message.content)

Hello! How can I help you today?


In [114]:
data_base = {'karachi to lahore':'plane name = Pk.2003 ,date=21 feb ,time = 11am','lahore to karachi':"plane name = pk.3000, date=22 feb,time=9am"}
def flight_detailes(Travelling):
    print(f'Tool calling for {Travelling}')
    plane_name = data_base.get(Travelling['Travelling'],"Unknow ticket")
    return f"Your travelling plane details is {plane_name}"



In [124]:
#funcation for booking 
def book_ticket(Travelling):
    print(f'Processing for booking ticket for {Travelling}')
    for key , value in data_base.items():
        if key == Travelling:
            #print("Your Ticket is booked")
            return f"your ticket booked and details of ticket is {data_base[Travelling]}"

In [126]:
print(book_ticket('karachi to lahore'))

Processing for booking ticket for karachi to lahore
your ticket booked and details of ticket is plane name = Pk.2003 ,date=21 feb ,time = 11am


In [107]:
tool = {
    'name':'flight_detailes',
    'description':'This funcation provide you the name of travelling plane',
    'parameters':{
        'type':'object',
        "properties":{
            'Travelling':{
                'type':'string',
                'description': "This is required parameter for getting the travelling plane name "
            },
        },
        'required':['Travelling']
    }
    
}

In [108]:
tools = {'type':'function','function':tool}

In [ ]:
system_message = """ You are an intelliegent Airport helper Agent  your tasked is to reply the user base on their input :
input are of two type: 
1st type = user : I went to travelled from karachi to lahore
assitent : reply wih plane name , date and time of avalibale plane
example:1 i went to go from islamabad to karachi?


answer : plane name = pk 1000 takeoff 20 feb at 9 pm 

example:2 i went to go from lahore to islamabad? 
answer: plane name pk 60000 takeoff 29 feb at 10 pm

2nd type = user : I went to travelled from lahore to karachi book the ticket for me
assistent : Congratulation i will  book the ticket for you , plane name pk.9000 , date = 20 feb , time =7am


 """

In [110]:
def handle_tool(content):
    content1 = content.tool_calls[0]
    if content1.function.name == 'flight_detailes':
        arg = json.loads(content1.function.arguments)
        result = flight_detailes(arg)
        response ={
            'role':'tool',
            'content':result,
            "tool_call_id": content1.id
        }
    return response    
    

In [ ]:
def chat_with_ai(message,history):
    history = [{'role':h['role'],'content':h['content']}for h in history]
    message = [{'role':'system','content':system_message}] + history + [{"role":'user',"content":message}]
    response = client.chat.completions.create(model='gemini-2.5-flash',messages=message,tools=tools)

    if response.choices[0].finish_reason=='tool_calls': 
        
        print('Tool calling')
        content = response.choices[0].message
        result = handle_tool(content)
        print('Comback from toolcall')
        message.append(content)
        message.append(result)
        print('Second call to LLm')
        response = client.chat.completions.create(model='gemini-2.5-flash',messages=message,tools=tools)

        
    return response.choices[0].message.content

In [112]:
import gradio as gr

In [113]:
gr.ChatInterface(fn=chat_with_ai,type='messages').launch()

* Running on local URL:  http://127.0.0.1:7884
* To create a public link, set `share=True` in `launch()`.


Tool calling
Tool calling for {'Travelling': 'lahore to karachi'}
Comback from toolcall
Second call to LLm
